# Car Classification using Stanford Cars Dataset

In [1]:
from cv2 import imread, imwrite, waitKey
from scipy import io as spio

##### 1. Load and match car metadata with images

In [2]:
car_metadata = spio.loadmat("cardata/cars_meta.mat")["class_names"]
car_train_raw = spio.loadmat("cardata/cars_train.mat")["annotations"]
car_test_raw = spio.loadmat("cardata/cars_test.mat")["annotations"]

#train/test metadata: [box x1, box y1, box x2, box y2, classnum, filename]

In [3]:
#minimal prep work required for the data as just need to load the metadata and match it up with the images
car_classes = [x[0] for x in car_metadata[0]]

car_train = {
    car[5][0]: car_classes[car[4][0][0]-1] for car in car_train_raw[0]
} #class gives num not index therefore -1

car_test = {
    car[5][0]: car_classes[car[4][0][0]-1] for car in car_test_raw[0]
}

#format: {"xxxxx.jpg": "Make Model Year"}

##### 2. Crop the car images to remove unnecessary background

In [5]:
#crop car images to only include the relevant bit, not background stuff
for car in car_train_raw[0]:
    x1, y1, x2, y2 = car[0][0][0], car[1][0][0], car[2][0][0], car[3][0][0]
    path = f"cardata/cars_train/{car[5][0]}"
    
    image = imread(path)
    crop = image[y1:y2, x1:x2]
    
    imwrite(path, crop)
    waitKey(0)
    
for car in car_test_raw[0]:
    x1, y1, x2, y2 = car[0][0][0], car[1][0][0], car[2][0][0], car[3][0][0]
    path = f"cardata/cars_test/{car[5][0]}"
    
    image = imread(path)
    crop = image[y1:y2, x1:x2]
    
    imwrite(path, crop)
    waitKey(0)

##### todo:
- normalize the data/rescale to small
- predict with epochs
- hyperparameter tuning

##### 3. Normalize the image data